In [1]:
# Update sklearn to prevent version mismatches
!conda install scikit-learn
!conda update scikit-learn
!conda install joblib 
!conda update joblib 

zsh:1: command not found: conda
zsh:1: command not found: conda
zsh:1: command not found: conda
zsh:1: command not found: conda


In [2]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [3]:
df = pd.read_csv("data_5hoods_datetime.csv")

In [4]:
#Select columns of interst
df = df[["Beat",
    "Occur Time",
    "Neighborhood",
    "Location Type",
    "Latitude",
    "Longitude",
    "Occur Year",
    "Occur Month",
    "Occur Day",
    "Occur DayofWeek",
    "UCR Literal"]]
#df.dropna(subset=['Location Type', 'Occur Time'])
df.dropna(subset=['Beat'], inplace=True)

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer 
y = df["UCR Literal"]
X = df.drop(columns=["UCR Literal"])

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder() 
X['Neighborhood']= le.fit_transform(X['Neighborhood']) 

# Create a Train Test Split

Use `UCR Literal` for the y values

In [6]:
#For example, if variable y is a binary categorical variable with values 0 and 1 and there are 25% of zeros and 75% of ones, stratify=y will make sure that your random split has 25% of 0's and 75% of 1's.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [7]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)#Compute the minimum and maximum to be used for later scaling.

X_train_scaled = X_scaler.transform(X_train)#Scale features of X_train according to feature_range.
X_test_scaled = X_scaler.transform(X_test)

In [8]:
#y_train
y_train.unique()

array(['LARCENY-FROM VEHICLE', 'ROBBERY-PEDESTRIAN', 'BURGLARY-NONRES',
       'AGG ASSAULT', 'LARCENY-NON VEHICLE', 'AUTO THEFT',
       'BURGLARY-RESIDENCE', 'ROBBERY-COMMERCIAL', 'HOMICIDE',
       'ROBBERY-RESIDENCE'], dtype=object)

In [9]:
X_train.head()

,Beat,Occur Time,Neighborhood,Location Type,Latitude,Longitude,Occur Year,Occur Month,Occur Day,Occur DayofWeek
36637,604.0,1630,4,21.0,33.76142,-84.36492,2013.0,12.0,16.0,0.0
52369,505.0,1230,3,13.0,33.77448,-84.38263,2017.0,9.0,3.0,6.0
9562,505.0,200,3,13.0,33.77454,-84.37885,2009.0,1.0,4.0,6.0
21715,511.0,930,2,99.0,33.75281,-84.39197,2011.0,1.0,25.0,1.0
40637,604.0,2030,4,18.0,33.75772,-84.37067,2014.0,10.0,11.0,5.0


# Pre-processing

Scale the data using LabelEncoder and MinMaxScaler

In [10]:
import tensorflow as tf
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import OneHotEncoder

In [11]:
label_encoder = LabelEncoder()# Should probably use OneHotEncoder instead
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)


In [12]:
y_train_categorical.shape[1]

10

In [13]:
y_train_categorical.shape

(41513, 10)

# Train the Model

In [14]:
X_test_scaled.shape[1]

10

In [15]:
y_train.nunique()

10

In [16]:
X_test_scaled.shape[1]

10

In [17]:
model = Sequential()
y_num_unique = y_train_categorical.shape[1]
X_num_col = X_test_scaled.shape[1]
model.add(Dense(units=100, activation='relu', input_dim=X_num_col))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=y_num_unique, activation='softmax'))

In [18]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               1100      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 12,210
Trainable params: 12,210
Non-trainable params: 0
_________________________________________________________________


In [20]:
# set early stopping as callback
callbacks = [EarlyStopping(monitor='val_loss', patience=2)]
#https://keras.rstudio.com/reference/fit.html
model.fit(
    X_train_scaled,
    y_train_categorical,
    callbacks=callbacks, #List of `keras.callbacks.Callback` instances.
    epochs=60, #Integer. Number of epochs to train the model.
    shuffle=True,#Boolean (whether to shuffle the training data before each epoch)
    verbose=1, #0 = silent, 1 = progress bar, 2 = one line per epoch.
    #view_metrics = 'auto'
)

Epoch 1/60
1298/1298 [==============================] - 1s 662us/step - loss: 1.5378 - accuracy: 0.44360s - loss: 1.5520 - accuracy
Epoch 2/60
1298/1298 [==============================] - 1s 604us/step - loss: 1.3593 - accuracy: 0.5470
Epoch 3/60
1298/1298 [==============================] - 1s 658us/step - loss: 1.2306 - accuracy: 0.5971
Epoch 4/60
1298/1298 [==============================] - 1s 614us/step - loss: 1.1960 - accuracy: 0.6055
Epoch 5/60
1298/1298 [==============================] - 1s 611us/step - loss: 1.1691 - accuracy: 0.6179
Epoch 6/60
1298/1298 [==============================] - 1s 650us/step - loss: 1.1603 - accuracy: 0.6190
Epoch 7/60
1298/1298 [==============================] - 1s 686us/step - loss: 1.1409 - accuracy: 0.6247
Epoch 8/60
1298/1298 [==============================] - 1s 640us/step - loss: 1.1305 - accuracy: 0.6276
Epoch 9/60
1298/1298 [==============================] - 1s 676us/step - loss: 1.1289 - accuracy: 0.6278
Epoch 10/60
1298/1298 [=============

1298/1298 [==============================] - 1s 628us/step - loss: 1.0542 - accuracy: 0.6511
Epoch 37/60
1298/1298 [==============================] - 1s 613us/step - loss: 1.0457 - accuracy: 0.6515
Epoch 38/60
1298/1298 [==============================] - 1s 583us/step - loss: 1.0398 - accuracy: 0.6510
Epoch 39/60
1298/1298 [==============================] - 1s 595us/step - loss: 1.0488 - accuracy: 0.6502
Epoch 40/60
1298/1298 [==============================] - 1s 605us/step - loss: 1.0421 - accuracy: 0.6521
Epoch 41/60
1298/1298 [==============================] - 1s 625us/step - loss: 1.0283 - accuracy: 0.6571
Epoch 42/60
1298/1298 [==============================] - 1s 628us/step - loss: 1.0335 - accuracy: 0.6556
Epoch 43/60
1298/1298 [==============================] - 1s 639us/step - loss: 1.0369 - accuracy: 0.6538
Epoch 44/60
1298/1298 [==============================] - 1s 673us/step - loss: 1.0358 - accuracy: 0.6556
Epoch 45/60
1298/1298 [==============================] - 1s 687us/s

In [21]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

639/639 - 0s - loss: 1.0742 - accuracy: 0.6500
Normal Neural Network - Loss: 1.0741653442382812, Accuracy: 0.6499902009963989


In [22]:
encoded_predictions = model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

/usr/local/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [23]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:5])}")

Predicted classes: ['LARCENY-NON VEHICLE' 'LARCENY-FROM VEHICLE' 'ROBBERY-PEDESTRIAN'
 'LARCENY-FROM VEHICLE' 'LARCENY-FROM VEHICLE']
Actual Labels: ['AUTO THEFT', 'LARCENY-FROM VEHICLE', 'ROBBERY-PEDESTRIAN', 'LARCENY-FROM VEHICLE', 'AUTO THEFT']


# Save the Model

In [52]:
import keras

In [53]:
model.save('idklol.h5')

In [71]:
reconstructed_model = keras.models.load_model("idklol.h5", compile=True)

In [79]:
predictions = reconstructed_model.predict_classes(X_test_scaled)
predictions.max()

8

In [73]:
reconstructed_model.fit(X_test_scaled, y_test_categorical)

639/639 [==============================] - 1s 628us/step - loss: 1.0650 - accuracy: 0.6529


In [66]:
reconstructed_model.predict(X_train_scaled)

array([[3.0293841e-02, 6.7632861e-02, 6.3553505e-02, ..., 1.0734848e-03,
        2.0735484e-02, 4.7921063e-03],
       [2.2807231e-02, 1.7564954e-01, 6.1571929e-03, ..., 6.4032042e-04,
        3.9618213e-02, 1.9632773e-04],
       [4.4283237e-02, 1.7627788e-01, 7.3368479e-03, ..., 3.1975575e-04,
        2.6015604e-01, 4.5181164e-03],
       ...,
       [2.2007364e-03, 9.4439974e-03, 8.3655536e-02, ..., 1.9120175e-03,
        3.1855304e-03, 7.3849129e-05],
       [1.7056884e-02, 1.1190084e-01, 2.0748987e-03, ..., 6.0069910e-04,
        5.9603840e-02, 1.0715683e-04],
       [9.1480598e-02, 3.2791860e-02, 6.7437552e-02, ..., 4.7420827e-03,
        2.3778971e-02, 8.7832976e-03]], dtype=float32)

In [58]:
X_train_scaled[0]

array([0.98046875, 0.67916667, 1.        , 0.20408163, 0.11941612,
       0.69499632, 0.4       , 1.        , 0.5       , 0.        ])

In [56]:
model.fit(X_train_scaled)

array([[2.3977187e-02, 9.8951019e-02, 5.1365569e-02, ..., 1.3484752e-03,
        2.2583222e-02, 1.6335694e-02],
       [3.7965398e-02, 1.7636184e-01, 1.1915312e-02, ..., 5.2821124e-03,
        5.4536778e-02, 1.2400452e-04],
       [6.8766430e-02, 1.4051265e-01, 3.5233505e-03, ..., 1.9072044e-04,
        3.9176857e-01, 2.1653804e-03],
       ...,
       [1.6988408e-03, 4.0298337e-03, 2.1566374e-02, ..., 9.8411564e-04,
        7.7598817e-03, 3.3084507e-04],
       [1.9098999e-02, 1.1138725e-01, 2.0083312e-03, ..., 7.0399925e-04,
        5.1027413e-02, 1.2447864e-04],
       [5.7612672e-02, 3.1706065e-02, 4.0009890e-02, ..., 5.0631310e-03,
        2.1363162e-02, 5.8322512e-03]], dtype=float32)